In [ ]:
from __future__ import annotations

from collections.abc import Iterable
from pathlib import Path

In [ ]:
type Position = tuple[int, int]

MAP: dict[Position, str] = {}

with Path("day12_input.txt").open() as file:
    for row, line in enumerate(file):
        for col, char in enumerate(line.strip()):
            MAP[(row, col)] = char

In [ ]:
def same_neighbors(pos: Position) -> Iterable[Position]:
    """Find the up/down left/right neighbors of a position."""
    row, col = pos
    candidates = [(row, col - 1), (row, col + 1), (row - 1, col), (row + 1, col)]
    for candidate in candidates:
        if (candidate in MAP) and (MAP[candidate] == MAP[pos]):
            yield candidate

# Part 1


In [ ]:
def find_region(
    start_pos: Position, region: set[Position] | None = None
) -> set[Position]:
    """Find all positions in the same region as start_pos."""
    if region is None:
        region = set()
    region.add(start_pos)
    for neighbor in same_neighbors(start_pos):
        if neighbor not in region:
            find_region(neighbor, region)
    return region

In [ ]:
regions = set()
answer = 0

for start_pos in MAP:
    if start_pos not in regions:
        region = find_region(start_pos)
        regions.update(region)
        perimeter = sum(4 - len(list(same_neighbors(pos))) for pos in region)
        # print(
        #     f"Region of {MAP[start_pos]} with size {len(region)} "
        #     f"and perimeter {perimeter}."
        # )
        answer += len(region) * perimeter

answer

# Part 2

The number of sides is also the same as the number of corners. A corner can be an
outside corner, or an inside corner.

Considering the upper-left corner, it can be a corner in two ways:

- Outside corner: If the cell does NOT have a neighbour to the left OR above it.
- Inside corner: If the cell DOES have neighbours to the left AND above it, AND the
  diagonal cell to the up-left is NOT a neighbour.

Thus: `upper_left = not (left or up) or (left and up and not up_left)`

```
AAA
.A.
AAA

```


In [ ]:
def num_corners(region: set[Position]) -> int:
    """Count the number of corners in a region."""
    corners = 0
    for pos in region:
        # Check for neighbors in all 8 directions
        left = (pos[0], pos[1] - 1) in region
        right = (pos[0], pos[1] + 1) in region
        up = (pos[0] - 1, pos[1]) in region
        down = (pos[0] + 1, pos[1]) in region
        up_left = (pos[0] - 1, pos[1] - 1) in region
        up_right = (pos[0] - 1, pos[1] + 1) in region
        down_left = (pos[0] + 1, pos[1] - 1) in region
        down_right = (pos[0] + 1, pos[1] + 1) in region

        # Check whether each potential corner is actually a corner
        upper_left = not (left or up) or (left and up and not up_left)
        upper_right = not (right or up) or (right and up and not up_right)
        lower_left = not (left or down) or (left and down and not down_left)
        lower_right = not (right or down) or (right and down and not down_right)

        # Add up the actual corners
        corners += upper_left + upper_right + lower_left + lower_right
    return corners

In [ ]:
regions = set()
answer = 0

for start_pos in MAP:
    if start_pos not in regions:
        region = find_region(start_pos)
        regions.update(region)
        corners = num_corners(region)
        # print(
        #     f"Region of {MAP[start_pos]} with size {len(region)} "
        #     f"and {corners} sides."
        # )
        answer += len(region) * corners

answer

.A
AA
A
